In [1]:
!wget https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar -O tabular-dl-tabr.tar.gz
!tar -xvf tabular-dl-tabr.tar.gz

--2024-11-16 11:51:41--  https://huggingface.co/datasets/puhsu/tabular-benchmarks/resolve/main/data.tar
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/1b/25/1b25d6e2556c827abfaf6ba9da6021338691cafc39fe9d77986955ae0a69243d/0d74e4b96febb48fbe4dd2d851f8638b5738354f82c3183f92adbd2abf2f256b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data.tar%3B+filename%3D%22data.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1732017101&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjAxNzEwMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy8xYi8yNS8xYjI1ZDZlMjU1NmM4MjdhYmZhZjZiYTlkYTYwMjEzMzg2OTFjYWZjMzlmZTlkNzc5ODY5NTVhZTBhNjkyNDNkLzBkNzRlNGI5NmZlYmI0OGZiZTRkZDJkODUxZjg2MzhiNTczODM1NGY4MmMzMTgzZjkyYWRiZDJhYmYyZjI1Nm

In [2]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.3 MB/s eta 0:00:00


In [3]:
!pip install psutil

In [4]:
import pandas as pd
import numpy as np
#import optuna
import torch.optim
import psutil

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import QuantileTransformer

In [ ]:
clf = TabNetClassifier()

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [5]:
X_train = np.load('/content/data/classif-num-medium-0-bank-marketing/X_num_train.npy')
y_train = np.load('/content/data/classif-num-medium-0-bank-marketing/Y_train.npy')

X_valid = np.load('/content/data/classif-num-medium-0-bank-marketing/X_num_val.npy')
y_valid = np.load('/content/data/classif-num-medium-0-bank-marketing/Y_val.npy')

X_test = np.load('/content/data/classif-num-medium-0-bank-marketing/X_num_test.npy')
y_test = np.load('/content/data/classif-num-medium-0-bank-marketing/Y_test.npy')

In [6]:
normalizer = QuantileTransformer(
            output_distribution='normal',
            n_quantiles=max(min(X_train.shape[0] // 30, 1000), 10),
            subsample=1_000_000_000,
            )
normalizer.fit_transform(X_train)

array([[-0.46086156, -0.31497777, -1.3420129 , ...,  1.0408198 ,
         0.7435598 ,  1.3172492 ],
       [-0.34431127, -0.97452503, -0.44386128, ...,  2.045391  ,
        -5.1993375 , -5.1993375 ],
       [-0.24806823,  0.10763446,  0.06142548, ..., -5.1993375 ,
        -5.1993375 , -5.1993375 ],
       ...,
       [-0.34431127, -0.02643349, -0.03069826, ..., -5.1993375 ,
         1.6072379 ,  1.3172492 ],
       [ 0.46086156, -0.10934963,  1.8431572 , ...,  0.15925325,
        -5.1993375 , -5.1993375 ],
       [-0.8489334 , -0.05350647,  0.81281084, ..., -5.1993375 ,
        -5.1993375 , -5.1993375 ]], dtype=float32)

In [ ]:
#Baseline
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
)

epoch 0  | loss: 0.65031 | train_accuracy: 0.58725 | valid_accuracy: 0.59559 |  0:00:02s
epoch 1  | loss: 0.56392 | train_accuracy: 0.50189 | valid_accuracy: 0.51891 |  0:00:02s
epoch 2  | loss: 0.53343 | train_accuracy: 0.52958 | valid_accuracy: 0.54832 |  0:00:03s
epoch 3  | loss: 0.51401 | train_accuracy: 0.54444 | valid_accuracy: 0.56513 |  0:00:03s
epoch 4  | loss: 0.51371 | train_accuracy: 0.55294 | valid_accuracy: 0.57983 |  0:00:04s
epoch 5  | loss: 0.49958 | train_accuracy: 0.55565 | valid_accuracy: 0.58088 |  0:00:05s
epoch 6  | loss: 0.4972  | train_accuracy: 0.542   | valid_accuracy: 0.56092 |  0:00:05s
epoch 7  | loss: 0.49496 | train_accuracy: 0.5493  | valid_accuracy: 0.55987 |  0:00:06s
epoch 8  | loss: 0.49125 | train_accuracy: 0.53579 | valid_accuracy: 0.55777 |  0:00:06s
epoch 9  | loss: 0.4872  | train_accuracy: 0.55227 | valid_accuracy: 0.57983 |  0:00:07s
epoch 10 | loss: 0.48062 | train_accuracy: 0.61831 | valid_accuracy: 0.65336 |  0:00:07s
epoch 11 | loss: 0.48

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
def objective(trial):
    params = {
        'n_d': trial.suggest_int('n_d', 8, 64),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        'momentum': trial.suggest_float('momentum', 0.01, 0.4),
    }

    model = TabNetClassifier(**params, n_a=params['n_d'])
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=['accuracy'], patience=10)

    # Evaluate model performance
    score = model.best_cost  # or any other metric

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params

[I 2024-07-06 03:18:06,577] A new study created in memory with name: no-name-31ec37ba-181a-4fad-95d7-92531a111062
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.66562 | val_0_accuracy: 0.47164 |  0:00:00s
epoch 1  | loss: 0.54388 | val_0_accuracy: 0.47584 |  0:00:01s
epoch 2  | loss: 0.51028 | val_0_accuracy: 0.53676 |  0:00:01s
epoch 3  | loss: 0.49443 | val_0_accuracy: 0.53992 |  0:00:02s
epoch 4  | loss: 0.49424 | val_0_accuracy: 0.54832 |  0:00:02s
epoch 5  | loss: 0.48309 | val_0_accuracy: 0.56828 |  0:00:03s
epoch 6  | loss: 0.47966 | val_0_accuracy: 0.55777 |  0:00:03s
epoch 7  | loss: 0.47289 | val_0_accuracy: 0.58088 |  0:00:04s
epoch 8  | loss: 0.47276 | val_0_accuracy: 0.57983 |  0:00:04s
epoch 9  | loss: 0.47303 | val_0_accuracy: 0.59139 |  0:00:05s
epoch 10 | loss: 0.47523 | val_0_accuracy: 0.6313  |  0:00:05s
epoch 11 | loss: 0.47391 | val_0_accuracy: 0.63235 |  0:00:06s
epoch 12 | loss: 0.47064 | val_0_accuracy: 0.64706 |  0:00:07s
epoch 13 | loss: 0.46446 | val_0_accuracy: 0.67332 |  0:00:07s
epoch 14 | loss: 0.46915 | val_0_accuracy: 0.69118 |  0:00:08s
epoch 15 | loss: 0.46434 | val_0_accuracy: 0.69643 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 03:18:29,620] Trial 0 finished with value: 0.805672268907563 and parameters: {'n_d': 27, 'n_steps': 3, 'gamma': 1.4649003372747087, 'n_independent': 4, 'n_shared': 4, 'momentum': 0.2997406682403858}. Best is trial 0 with value: 0.805672268907563.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.10912 | val_0_accuracy: 0.6313  |  0:00:00s
epoch 1  | loss: 0.77692 | val_0_accuracy: 0.4979  |  0:00:01s
epoch 2  | loss: 0.66526 | val_0_accuracy: 0.50945 |  0:00:02s
epoch 3  | loss: 0.78384 | val_0_accuracy: 0.51471 |  0:00:03s
epoch 4  | loss: 0.6475  | val_0_accuracy: 0.53782 |  0:00:04s
epoch 5  | loss: 0.61414 | val_0_accuracy: 0.47164 |  0:00:05s
epoch 6  | loss: 0.64578 | val_0_accuracy: 0.65021 |  0:00:06s
epoch 7  | loss: 0.5836  | val_0_accuracy: 0.57458 |  0:00:07s
epoch 8  | loss: 0.53941 | val_0_accuracy: 0.65441 |  0:00:08s
epoch 9  | loss: 0.51683 | val_0_accuracy: 0.69958 |  0:00:09s
epoch 10 | loss: 0.51213 | val_0_accuracy: 0.67962 |  0:00:10s
epoch 11 | loss: 0.49909 | val_0_accuracy: 0.67017 |  0:00:11s
epoch 12 | loss: 0.5197  | val_0_accuracy: 0.65336 |  0:00:12s
epoch 13 | loss: 0.5045  | val_0_accuracy: 0.70903 |  0:00:13s
epoch 14 | loss: 0.49926 | val_0_accuracy: 0.73109 |  0:00:13s
epoch 15 | loss: 0.48326 | val_0_accuracy: 0.72584 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 03:19:11,055] Trial 1 finished with value: 0.8046218487394958 and parameters: {'n_d': 36, 'n_steps': 8, 'gamma': 1.758750612898449, 'n_independent': 5, 'n_shared': 1, 'momentum': 0.06385752439320777}. Best is trial 0 with value: 0.805672268907563.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

epoch 0  | loss: 0.95329 | val_0_accuracy: 0.48634 |  0:00:00s
epoch 1  | loss: 0.66339 | val_0_accuracy: 0.49055 |  0:00:01s
epoch 2  | loss: 0.58347 | val_0_accuracy: 0.51996 |  0:00:02s
epoch 3  | loss: 0.57054 | val_0_accuracy: 0.56303 |  0:00:03s
epoch 4  | loss: 0.53782 | val_0_accuracy: 0.52416 |  0:00:04s
epoch 5  | loss: 0.51814 | val_0_accuracy: 0.52731 |  0:00:04s
epoch 6  | loss: 0.50829 | val_0_accuracy: 0.55042 |  0:00:05s
epoch 7  | loss: 0.50156 | val_0_accuracy: 0.55777 |  0:00:06s
epoch 8  | loss: 0.49896 | val_0_accuracy: 0.63866 |  0:00:07s
epoch 9  | loss: 0.50014 | val_0_accuracy: 0.64706 |  0:00:08s
epoch 10 | loss: 0.49121 | val_0_accuracy: 0.64706 |  0:00:08s
epoch 11 | loss: 0.48088 | val_0_accuracy: 0.61765 |  0:00:09s
epoch 12 | loss: 0.47857 | val_0_accuracy: 0.64496 |  0:00:10s
epoch 13 | loss: 0.48728 | val_0_accuracy: 0.69538 |  0:00:11s
epoch 14 | loss: 0.48729 | val_0_accuracy: 0.68592 |  0:00:12s
epoch 15 | loss: 0.48959 | val_0_accuracy: 0.69013 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 04:09:35,271] Trial 98 finished with value: 0.8130252100840336 and parameters: {'n_d': 26, 'n_steps': 5, 'gamma': 1.5256922723904967, 'n_independent': 3, 'n_shared': 5, 'momentum': 0.3977905098853236}. Best is trial 81 with value: 0.8319327731092437.
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.34668 | val_0_accuracy: 0.53782 |  0:00:00s
epoch 1  | loss: 0.59891 | val_0_accuracy: 0.56197 |  0:00:00s
epoch 2  | loss: 0.53673 | val_0_accuracy: 0.58718 |  0:00:01s
epoch 3  | loss: 0.50922 | val_0_accuracy: 0.60924 |  0:00:01s
epoch 4  | loss: 0.49985 | val_0_accuracy: 0.61345 |  0:00:01s
epoch 5  | loss: 0.48925 | val_0_accuracy: 0.59244 |  0:00:02s
epoch 6  | loss: 0.48339 | val_0_accuracy: 0.59349 |  0:00:02s
epoch 7  | loss: 0.48456 | val_0_accuracy: 0.60399 |  0:00:02s
epoch 8  | loss: 0.47622 | val_0_accuracy: 0.6166  |  0:00:03s
epoch 9  | loss: 0.47455 | val_0_accuracy: 0.62815 |  0:00:03s
epoch 10 | loss: 0.47596 | val_0_accuracy: 0.62395 |  0:00:04s
epoch 11 | loss: 0.47287 | val_0_accuracy: 0.66071 |  0:00:04s
epoch 12 | loss: 0.46869 | val_0_accuracy: 0.67752 |  0:00:04s
epoch 13 | loss: 0.47449 | val_0_accuracy: 0.69223 |  0:00:05s
epoch 14 | loss: 0.47312 | val_0_accuracy: 0.67857 |  0:00:05s
epoch 15 | loss: 0.46893 | val_0_accuracy: 0.70378 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2024-07-06 04:09:55,816] Trial 99 finished with value: 0.8109243697478992 and parameters: {'n_d': 20, 'n_steps': 4, 'gamma': 1.0661642960371185, 'n_independent': 2, 'n_shared': 1, 'momentum': 0.11741906848438047}. Best is trial 81 with value: 0.8319327731092437.


epoch 56 | loss: 0.44127 | val_0_accuracy: 0.80672 |  0:00:20s

Early stopping occurred at epoch 56 with best_epoch = 46 and best_val_0_accuracy = 0.81092


In [ ]:
best_params

{'n_d': 29,
 'n_steps': 4,
 'gamma': 1.0642961521028647,
 'n_independent': 2,
 'n_shared': 1,
 'momentum': 0.07192877371900636}

In [7]:
best_params = {'n_d': 29,
 'n_steps': 4,
 'gamma': 1.0642961521028647,
 'n_independent': 2,
 'n_shared': 1,
 'momentum': 0.07192877371900636}

In [8]:
from pytorch_tabnet.callbacks import Callback

def print_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"System RAM Usage: {mem_info.rss / (1024 ** 2):.2f} MB")
    if torch.cuda.is_available():
      print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / (1024 ** 2):.2f} MB")
      print(f"Cached GPU memory: {torch.cuda.memory_reserved() / (1024 ** 2):.2f} MB")

class MemoryUsageCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print_memory_usage()

In [9]:
# Train final model using best hyperparameters
best_model = TabNetClassifier(
    **best_params,
    n_a=best_params['n_d'],
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2)
)

best_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['accuracy'],
    callbacks=[MemoryUsageCallback()]
)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.92142 | train_accuracy: 0.4869  | valid_accuracy: 0.4958  |  0:00:06s
System RAM Usage: 1176.79 MB
Allocated GPU memory: 17.74 MB
Cached GPU memory: 46.00 MB
epoch 1  | loss: 0.53697 | train_accuracy: 0.48649 | valid_accuracy: 0.50525 |  0:00:07s
System RAM Usage: 1176.79 MB
Allocated GPU memory: 17.74 MB
Cached GPU memory: 46.00 MB
epoch 2  | loss: 0.50458 | train_accuracy: 0.49554 | valid_accuracy: 0.52311 |  0:00:08s
System RAM Usage: 1176.79 MB
Allocated GPU memory: 17.74 MB
Cached GPU memory: 46.00 MB
epoch 3  | loss: 0.48853 | train_accuracy: 0.53633 | valid_accuracy: 0.57143 |  0:00:09s
System RAM Usage: 1176.79 MB
Allocated GPU memory: 17.74 MB
Cached GPU memory: 46.00 MB
epoch 4  | loss: 0.47964 | train_accuracy: 0.51121 | valid_accuracy: 0.53676 |  0:00:09s
System RAM Usage: 1176.79 MB
Allocated GPU memory: 17.74 MB
Cached GPU memory: 46.00 MB
epoch 5  | loss: 0.47638 | train_accuracy: 0.52593 | valid_accuracy: 0.55567 |  0:00:10s
System RAM Usage: 1177.20 

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.8109243697478992
Test AUC: 0.8602697571990754


In [11]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

y_proba = best_model.predict_proba(X_test)
test_auc = roc_auc_score(y_score=y_proba[:,1], y_true=y_test)

y_proba_valid = best_model.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=y_proba_valid[:,1], y_true=y_valid)

print("Best Valid Score:", best_model.best_cost)
print("Test AUC:", test_auc)

Best Valid Score: 0.8109243697478992
Test AUC: 0.8602697571990754


In [12]:
fpr, tpr, thresholds = roc_curve(y_score=y_proba[:,1], y_true=y_test)

roc_data = pd.DataFrame({
    'fpr': fpr,
    'tpr': tpr
})

roc_data.to_csv('fpr_tpr_values.csv', index=False)

In [ ]:
roc_data

,False Positive Rate,True Positive Rate
0,0.000000,0.000000
1,0.000000,0.000892
2,0.000000,0.014273
3,0.000908,0.014273
4,0.000908,0.023194
...,...,...
665,0.892825,0.998216
666,0.892825,0.999108
667,0.929155,0.999108
668,0.929155,1.000000


In [ ]:
preds = best_model.predict(X_test)

print(classification_report(y_true=y_test, y_pred=preds))

              precision    recall  f1-score   support

           0       0.79      0.76      0.78      1101
           1       0.78      0.80      0.79      1121

    accuracy                           0.78      2222
   macro avg       0.79      0.78      0.78      2222
weighted avg       0.79      0.78      0.78      2222

